In [2]:
import numpy as np
from scipy.stats.mstats import mode
from copy import deepcopy
import pandas as pd
import time
%matplotlib inline
from helper import *
from IDWmodel import *

In [110]:
# plot how much data for each sensor is missing in %
def compute_missing_entries_per_day(df, num_days):
    missing_per_day = {}
    for day in xrange(1, num_days + 1):
        print 'processing day {} ...'.format(day)
        daydict = {}
        for i in xrange(56):
            sid = 'S'+str(i+1)
            df_day = df[df['day'] == day]
            idxs = df_day[sid] == -1
            daydict[sid] = np.sum(idxs * 1.) / (1. * df_day.count()[0])
        missing_per_day[day] = daydict
    return missing_per_day

def wavg(val_col_name,wt_col_name):
    def inner(group):
        val = (group[val_col_name] * group[wt_col_name]).sum() / group[wt_col_name].sum()
        return np.round(val)
    inner.__name__ = 'wgt_avg'
    return inner    

# ### Building a reference table with average daily value of the sensor
# specify with num_minutes over how many minutes the time should be bucketed
def build_weighted_time_table(df_train, num_minutes=5):
    # different averaging does not seem to have an effect...
    df_train['day'] =  df_train['time'] // 10000
    df_train['day_time'] = (df_train['time'] % (100 * 100)) // 100 * 100 + ((df_train['time'] % 100) // num_minutes) * num_minutes
    
    num_days = int(df_train.day.max())
    missing_per_day = compute_missing_entries_per_day(df_train, num_days)
    
    # augment using the missing entries
    for i in xrange(56):
        sid = 'S'+str(i+1)
        wid = 'w'+str(i+1)
        df_train[wid] = 0.
        for day in xrange(1, num_days+1):
            df_train.loc[df_train['day'] == day, wid] = 1. - missing_per_day[day][sid]
    
    # normalize weights, so they sum up to 1!
    # assert here that there are num_days rows!
    df_normalizer = df_train[df_train['time'] % 10000 == 0]
    assert(df_normalizer.count()[0] == num_days)

    for i in xrange(56):
        wid = 'w'+str(i+1)
        df_train[wid] /= df_normalizer.sum()[wid] * 1.
        
        
    # Initializing the dataframe
    # Update: rounding the value is done in the wavg function, should be maybe adjusted!
    col_name = 'S1'
    wid = 'w1'
    df_day_avg_values = pd.DataFrame(df_train[[col_name, 'day_time', wid]][df_train[col_name] != -1].groupby('day_time').apply(wavg(col_name, wid)), columns=[col_name])
    df_day_avg_values
    for i in xrange(1, 56):
        col_name = 'S'+str(i+1)
        wid = 'w'+str(i+1)
        print 'processing '+col_name
        df_day_frame = pd.DataFrame(df_train[[col_name, 'day_time', wid]][df_train[col_name] != -1].groupby('day_time').apply(wavg(col_name, wid)), columns=[col_name])
        df_day_avg_values = df_day_avg_values.join(df_day_frame)
        
    return df_day_avg_values

In [111]:
df_train = load_train_data()

In [112]:
df_train = load_train_data()
df_IDWmodel = pd.read_csv('data/IDWmodel_train.csv')
df_day_avg_values = build_weighted_time_table(df_train)

processing day 1 ...
processing day 2 ...
processing day 3 ...
processing day 4 ...
processing day 5 ...
processing day 6 ...
processing day 7 ...
processing day 8 ...
processing day 9 ...
processing day 10 ...
processing day 11 ...
processing day 12 ...
processing day 13 ...
processing day 14 ...
processing day 15 ...
processing day 16 ...
processing day 17 ...
processing day 18 ...
processing day 19 ...
processing day 20 ...
processing day 21 ...
processing day 22 ...
processing day 23 ...
processing day 24 ...
processing day 25 ...
processing day 26 ...
processing day 27 ...
processing day 28 ...
processing day 29 ...
processing day 30 ...
processing S2
processing S3
processing S4
processing S5
processing S6
processing S7
processing S8
processing S9
processing S10
processing S11
processing S12
processing S13
processing S14
processing S15
processing S16
processing S17
processing S18
processing S19
processing S20
processing S21
processing S22
processing S23
processing S24
processing S